# Modèle de Prédiction des Maladies Cardiaques

Ce notebook implémente et compare plusieurs algorithmes de machine learning pour la prédiction des maladies cardiaques à partir du dataset Heart Disease.

---

## 1. Importation des Bibliothèques

In [13]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
import joblib
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split

## 2. Chargement des Données

### Dataset Heart Disease
- **Structure** :
  - **Features (X)** : 13 colonnes
  - **Target (y)** : Colonne `target` (0 = sain, 1 = malade)
- **Objectif** : La prédiction de la présence de maladie cardiaque (risque de la maladie)

In [14]:
df = pd.read_csv('dataset/heart.csv')
X = df.drop('target', axis=1)
y = df['target']

### Division des donneés
- **80% pour l'entraînement** : Pour apprendre les patterns
- **20% pour le test** : Pour évaluer la performance
- **Configuration** :
  - `test_size=0.2` : 20% pour le test
  - `random_state=42` : Reproductibilité
  - `stratify=y` : Préservation de la distribution des classes

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

### Standardisation des Features
- Certains algorithmes (SVM, Régression Logistique) sont sensibles aux échelles
- Pour cela nous utilisons "StandardScaler:
  - Étape 1 : Centrer (mettre la moyenne à 0)
  - Étape 2 : Réduire (mettre écart-type à 1)
- **Approche** :
  - Entraînement du scaler sur `X_train` seulement
  - Application sur `X_train` et `X_test`
- **Objectif** : Améliorer la convergence et la performance

In [16]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## 3. Modèles et entraînement 

### Régression Logistique

- **Type** : Classification linéaire
- **Paramètres** :
  - `random_state=42` : Reproductibilité
  - `max_iter=1000` : Assurance de convergence
- **Avantages** :
  - Simple et interprétable
  - Bonne performance avec données linéairement séparables
- **Prérequis** : Données normalisées

In [17]:
lr_model = LogisticRegression(random_state=42, max_iter=1000)
lr_model.fit(X_train_scaled, y_train)
y_pred_lr = lr_model.predict(X_test_scaled)
accuracy_lr = accuracy_score(y_test, y_pred_lr)

### SVM avec Noyau Linéaire
- Trouver l'hyperplan optimal séparant les classes deux classes
- **Noyau** : Linéaire 
- **Accuracy** : 81.46% 
- **Caractéristiques** :
  - Moins de risques de surapprentissage
  - Paramètres simples à ajuster

In [18]:
svm_linear = SVC(kernel='linear', random_state=42)
svm_linear.fit(X_train_scaled, y_train)
y_pred_linear = svm_linear.predict(X_test_scaled)
accuracy_linear = accuracy_score(y_test, y_pred_linear)
print(f"SVM Linéaire - Accuracy: {accuracy_linear:.4f}")

SVM Linéaire - Accuracy: 0.8146


### SVM avec Noyau Polynomial (Degré 3)
- Transforme l'espace d'entrée en dimensions supérieures
- **Degré** : 3 (frontières cubiques)
- **Accuracy** : 92.68%
- **Avantages** :
  - Flexible avec différents degrés
  - Bon compromis complexité/performance

In [19]:
svm_poly = SVC(kernel='poly', degree=3, random_state=42)
svm_poly.fit(X_train_scaled, y_train)
y_pred_poly = svm_poly.predict(X_test_scaled)
accuracy_poly = accuracy_score(y_test, y_pred_poly)
print(f"SVM Polynomial (deg 3) - Accuracy: {accuracy_poly:.4f}")

SVM Polynomial (deg 3) - Accuracy: 0.9268


### SVM avec Noyau Radial Basis Function
- Utilise des fonctions gaussiennes
- **Caractéristiques** :
  - Très flexible
  - Bon pour données complexes
  - Nécessite un réglage fin
- **Accuracy** : 92.68%
- **Comparaison** : Similaire au polynomial sur ce dataset

In [20]:
svm_rbf = SVC(kernel='rbf', random_state=42)
svm_rbf.fit(X_train_scaled, y_train)
y_pred_rbf = svm_rbf.predict(X_test_scaled)
accuracy_rbf = accuracy_score(y_test, y_pred_rbf)
print(f"SVM RBF - Accuracy: {accuracy_rbf:.4f}")

SVM RBF - Accuracy: 0.9268


### Ensemble d'Arbres de Décision (Random Forest)
- Combinaison de multiples arbres
- **Paramètres clés** :
  - `n_estimators=100` : 100 arbres
  - `max_depth=5` : Limite la profondeur
  - `max_features='sqrt'` : Sélection aléatoire de features
  - `n_jobs=-1` : Parallélisation sur tous les cœurs
- **Avantages** :
  - Réduction du surapprentissage
  - Importance des features
- **Accuracy** : 92.68%

In [21]:
rf_classifier = RandomForestClassifier(
    n_estimators=100,     
    max_depth=5,           
    min_samples_split=2,   
    min_samples_leaf=1,    
    max_features='sqrt',   
    random_state=42,
    n_jobs=-1              
)

rf_classifier.fit(X_train, y_train)

predictions = rf_classifier.predict(X_test)

accuracy_rf = accuracy_score(y_test, predictions)
print(f"Random Forest - Accuracy: {accuracy_rf:.4f}")

Random Forest - Accuracy: 0.9268


### Tableau des Performances
| Modèle | Accuracy | Performance |
|--------|----------|-------------|
| Régression Logistique | 80.98% | Moyenne |
| SVM Linéaire | 81.46% | Moyenne |
| SVM Polynomial | **92.68%** | Excellente |
| SVM RBF | **92.68%** | Excellente |
| Random Forest | **92.68%** | Excellente |

### Analyse des Résultats
- **Tendance** : Modèles non-linéaires > Modèles linéaires
- **Performance maximale** : 92.68% atteinte par 3 modèles
- **Stabilité** : Modèles ensemble plus robustes

In [22]:
print(f"\nCOMPARISON:")
print(f"Logistic Regression: {accuracy_lr:.4f}")
print(f"SVM Linear: {accuracy_linear:.4f}")
print(f"SVM Polynomial: {accuracy_poly:.4f}")
print(f"SVM RBF: {accuracy_rbf:.4f}")
print(f"Random Forest: {accuracy_rf:.4f}")


COMPARISON:
Logistic Regression: 0.8098
SVM Linear: 0.8146
SVM Polynomial: 0.9268
SVM RBF: 0.9268
Random Forest: 0.9268


### Processus de Sélection
1. **Collecte** : Tous les modèles et leurs performances
2. **Évaluation** : Comparaison par accuracy
3. **Décision** : Sélection du meilleur
4. **Résultat** : SVM Polynomial sélectionné

### Caractéristiques du Modèle Gagnant
- **Nom** : SVM Polynomial (Degré 3)
- **Accuracy** : 92.68%
- **Type** : Machine à vecteurs de support
- **Noyau** : Polynomial

In [23]:
models = {
    'Logistic Regression': (lr_model, accuracy_lr),
    'SVM Linear': (svm_linear, accuracy_linear),
    'SVM Polynomial': (svm_poly, accuracy_poly),
    'SVM RBF': (svm_rbf, accuracy_rbf),
    'Random Forest': (rf_classifier,accuracy_rf)
}

best_model_name = max(models, key=lambda x: models[x][1])
best_model, best_accuracy = models[best_model_name]

print(f"\nBest Model: {best_model_name} - Accuracy: {best_accuracy:.4f}")



Best Model: SVM Polynomial - Accuracy: 0.9268


### Fichiers Exportés
1. **`heart_model.pkl`** : Modèle entraîné (SVM Polynomial)
2. **`scaler.pkl`** : Normaliseur pour nouvelles données

### Utilisation
- **Chargement** : `joblib.load()` pour réutiliser
- **Prédiction** : Données → Normalisation → Modèle → Résultat
- **Maintenance** : Mise à jour possible avec nouvelles données

In [24]:
joblib.dump(best_model, 'model/heart_model.pkl')
joblib.dump(scaler, 'model/scaler.pkl')
print("The model and the scaler are saved!")

The model and the scaler are saved!
